# Building Permits Dataset: Data Preparation

This notebook documents the comprehensive data preparation process for the Building Permits dataset. We'll follow a structured approach to clean, transform, and prepare the data for analysis and modeling.

## 1. Import Libraries

In [10]:
# Import essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
import warnings

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('deep')
%matplotlib inline

## 2. Load Dataset

We'll load the Building Permits dataset. Based on our previous analysis, we know there are mixed-type columns (columns 22 and 32), so we'll use `low_memory=False` to handle them properly.

In [5]:
# Load the dataset with appropriate parameters
df = pd.read_csv("datasets/Building_Permits.csv", low_memory=False)

# Display basic information
print(f"Dataset Shape: {df.shape}")
df.head()

Dataset Shape: (198900, 43)


,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,Unit,Unit Suffix,Description,Current Status,Current Status Date,Filed Date,Issued Date,Completed Date,First Construction Document Date,Structural Notification,Number of Existing Stories,Number of Proposed Stories,Voluntary Soft-Story Retrofit,Fire Only Permit,Permit Expiration Date,Estimated Cost,Revised Cost,Existing Use,Existing Units,Proposed Use,Proposed Units,Plansets,TIDF Compliance,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,NaN,Ellis,St,NaN,NaN,"ground fl facade: to erect illuminated, electr...",expired,12/21/2017,05/06/2015,11/09/2015,NaN,11/09/2015,NaN,6.00,NaN,NaN,NaN,11/03/2016,4000.00,4000.00,tourist hotel/motel,143.00,NaN,NaN,2.00,NaN,3.00,constr type 3,NaN,NaN,NaN,3.00,Tenderloin,94102.00,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,NaN,Geary,St,0.00,NaN,remove (e) awning and associated signs.,issued,08/03/2017,04/19/2016,08/03/2017,NaN,08/03/2017,NaN,7.00,NaN,NaN,NaN,12/03/2017,1.00,500.00,tourist hotel/motel,NaN,NaN,NaN,2.00,NaN,3.00,constr type 3,NaN,NaN,NaN,3.00,Tenderloin,94102.00,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,NaN,Pacific,Av,NaN,NaN,installation of separating wall,withdrawn,09/26/2017,05/27/2016,NaN,NaN,NaN,NaN,6.00,6.00,NaN,NaN,NaN,20000.00,NaN,retail sales,39.00,retail sales,39.00,2.00,NaN,1.00,constr type 1,1.00,constr type 1,NaN,3.00,Russian Hill,94109.00,"(37.7946573324287, -122.42232562979227)",1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,NaN,Pacific,Av,0.00,NaN,repair dryrot & stucco at front of bldg.,complete,07/24/2017,11/07/2016,07/18/2017,07/24/2017,07/18/2017,NaN,2.00,2.00,NaN,NaN,07/13/2018,2000.00,2000.00,1 family dwelling,1.00,1 family dwelling,1.00,2.00,NaN,5.00,wood frame (5),5.00,wood frame (5),NaN,3.00,Nob Hill,94109.00,"(37.79595867909168, -122.41557405519474)",1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,NaN,Market,St,NaN,NaN,demolish retail/office/commercial 3-story buil...,issued,12/01/2017,11/28/2016,12/01/2017,NaN,11/20/2017,NaN,3.00,NaN,NaN,NaN,12/01/2018,100000.00,100000.00,retail sales,NaN,NaN,NaN,2.00,NaN,3.00,constr type 3,NaN,NaN,NaN,6.00,Tenderloin,94102.00,"(37.78315261897309, -122.40950883997789)",144548169992


## 3. Initial Dataset Inspection

Let's get a deeper understanding of our dataset by checking data types, missing values, and basic statistics.

In [6]:
# Check data types and missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198900 entries, 0 to 198899
Data columns (total 43 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   Permit Number                           198900 non-null  object 
 1   Permit Type                             198900 non-null  int64  
 2   Permit Type Definition                  198900 non-null  object 
 3   Permit Creation Date                    198900 non-null  object 
 4   Block                                   198900 non-null  object 
 5   Lot                                     198900 non-null  object 
 6   Street Number                           198900 non-null  int64  
 7   Street Number Suffix                    2216 non-null    object 
 8   Street Name                             198900 non-null  object 
 9   Street Suffix                           196132 non-null  object 
 10  Unit                                    2947

In [11]:
# Check basic statistics for numeric columns
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Permit Type,198900.0000,7.5223,1.4575,1.0000,8.0000,8.0000,8.0000,8.0000
Street Number,198900.0000,1121.7289,1135.7689,0.0000,235.0000,710.0000,1700.0000,8400.0000
Unit,29479.0000,78.5172,326.9813,0.0000,0.0000,0.0000,1.0000,6004.0000
Number of Existing Stories,156116.0000,5.7058,8.6135,0.0000,2.0000,3.0000,4.0000,78.0000
Number of Proposed Stories,156032.0000,5.7450,8.6133,0.0000,2.0000,3.0000,4.0000,78.0000
Estimated Cost,160834.0000,168955.4433,3630385.9537,1.0000,3300.0000,11000.0000,35000.0000,537958646.0000
Revised Cost,192834.0000,132856.1865,3584902.5920,0.0000,1.0000,7000.0000,28707.5000,780500000.0000
Existing Units,147362.0000,15.6662,74.4763,0.0000,1.0000,1.0000,4.0000,1907.0000
Proposed Units,147989.0000,16.5110,75.2204,0.0000,1.0000,2.0000,4.0000,1911.0000
Plansets,161591.0000,1.2747,22.4073,0.0000,0.0000,2.0000,2.0000,9000.0000


In [12]:
# Count missing values in each column
missing_values = df.isna().sum()
missing_percent = (missing_values / len(df)) * 100

# Create a dataframe with missing value information
missing_info = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage': missing_percent,
})

# Display columns with missing values (more than 0)
missing_info[missing_info['Missing Values'] > 0].sort_values('Percentage', ascending=False)

,Missing Values,Percentage
TIDF Compliance,198898,99.9990
Voluntary Soft-Story Retrofit,198865,99.9824
Unit Suffix,196939,99.0141
Street Number Suffix,196684,98.8859
Site Permit,193541,97.3057
Structural Notification,191978,96.5199
Fire Only Permit,180073,90.5344
Unit,169421,85.1790
Completed Date,101709,51.1357
Permit Expiration Date,51880,26.0835


## 4. Examining Mixed-Type Columns

From our previous analysis, we identified columns 22 and 32 as having mixed types. Let's investigate these specifically.

In [ ]:
# Get the column names at positions 22 and 32
col_22 = df.columns[22]
col_32 = df.columns[32]
print(f"Column 22: {col_22}")
print(f"Column 32: {col_32}")

In [ ]:
# Examine column 22
print("\nColumn 22 data type:", df[col_22].dtype)
print("\nSample values:")
print(df[col_22].head())
print("\nUnique values (first 20):")
print(df[col_22].unique()[:20])
print(f"\nTotal unique values: {df[col_22].nunique()}")

In [ ]:
# Examine column 32
print("\nColumn 32 data type:", df[col_32].dtype)
print("\nSample values:")
print(df[col_32].head())
print("\nUnique values (first 20):")
print(df[col_32].unique()[:20])
print(f"\nTotal unique values: {df[col_32].nunique()}")

## 5. Data Cleaning

Based on our inspection, we'll now clean the dataset by addressing missing values, handling mixed types, and correcting data inconsistencies.

### 5.1 Fixing Mixed-Type Columns

We'll convert the problematic columns to appropriate data types.

In [ ]:
# Create a copy of the dataframe to avoid modifying the original
df_clean = df.copy()

# Convert the mixed-type columns to string for consistency
df_clean[col_22] = df_clean[col_22].astype(str)
df_clean[col_32] = df_clean[col_32].astype(str)

# Replace 'nan' strings with actual NaN values
df_clean.replace('nan', np.nan, inplace=True)
df_clean.replace('None', np.nan, inplace=True)

### 5.2 Converting Date Columns

In [ ]:
# Identify date columns (this is an example - adjust based on your actual date columns)
date_columns = [col for col in df_clean.columns if 'DATE' in col.upper() or 'FILED' in col.upper()]
print(f"Potential date columns: {date_columns}")

# Convert date columns to datetime format
for col in date_columns:
    try:
        df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
        print(f"Successfully converted {col} to datetime")
    except Exception as e:
        print(f"Could not convert {col}: {e}")

### 5.3 Handling Missing Values

We'll handle missing values based on the context of each column.

In [ ]:
# Count missing values after initial cleaning
missing_after_cleaning = df_clean.isnull().sum()
missing_percent_after = (missing_after_cleaning / len(df_clean)) * 100

missing_info_after = pd.DataFrame({
    'Missing Values': missing_after_cleaning,
    'Percentage': missing_percent_after
})

# Display columns with missing values (more than 0)
missing_cols = missing_info_after[missing_info_after['Missing Values'] > 0].sort_values('Percentage', ascending=False)
missing_cols

In [ ]:
# Strategy for handling missing values
# 1. Drop columns with too many missing values (e.g., >90%)
high_missing_cols = missing_cols[missing_cols['Percentage'] > 90].index.tolist()
print(f"Columns with >90% missing values (to be dropped): {high_missing_cols}")
df_clean.drop(columns=high_missing_cols, inplace=True)

# 2. For numerical columns with moderate missing values, fill with median
numeric_cols = df_clean.select_dtypes(include=['int64', 'float64']).columns
for col in numeric_cols:
    if df_clean[col].isnull().sum() > 0:
        df_clean[col].fillna(df_clean[col].median(), inplace=True)
        print(f"Filled missing values in {col} with median")

# 3. For categorical columns, fill with most frequent value or 'Unknown'
categorical_cols = df_clean.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if df_clean[col].isnull().sum() > 0:
        if df_clean[col].nunique() < 10:  # For low cardinality columns
            df_clean[col].fillna(df_clean[col].mode()[0], inplace=True)
            print(f"Filled missing values in {col} with mode")
        else:  # For high cardinality columns
            df_clean[col].fillna('Unknown', inplace=True)
            print(f"Filled missing values in {col} with 'Unknown'")

# 4. For datetime columns, keep nulls as they might be meaningful
# (we can filter them out later if needed)

## 6. Outlier Detection and Treatment

We'll use the Z-Score method to identify potential outliers in numerical columns.

In [9]:
def detect_outliers_zscore(df, column, threshold=3):
    """Detect outliers using Z-score method"""
    z_scores = np.abs((df[column] - df[column].mean()) / df[column].std())
    outliers = df[z_scores > threshold]
    return outliers, len(outliers)

# Check outliers in numerical columns
numeric_cols = df_clean.select_dtypes(include=['int64', 'float64']).columns

for col in numeric_cols:
    if df_clean[col].nunique() > 1:  # Only check columns with variation
        outliers, count = detect_outliers_zscore(df_clean, col)
        print(f"Column: {col}, Outliers: {count} ({count/len(df_clean)*100:.2f}%)")
        
        # If the percentage of outliers is small, we can cap them
        if 0 < count/len(df_clean)*100 < 5:
            lower_bound = df_clean[col].mean() - 3 * df_clean[col].std()
            upper_bound = df_clean[col].mean() + 3 * df_clean[col].std()
            df_clean[col] = df_clean[col].clip(lower=lower_bound, upper=upper_bound)
            print(f"  - Capped outliers in {col}")

NameError: name 'df_clean' is not defined

## 7. Feature Engineering

Let's create some useful features from the existing data.

In [ ]:
# 7.1 Extract date components
date_columns = df_clean.select_dtypes(include=['datetime64']).columns

for col in date_columns:
    df_clean[f"{col}_year"] = df_clean[col].dt.year
    df_clean[f"{col}_month"] = df_clean[col].dt.month
    df_clean[f"{col}_quarter"] = df_clean[col].dt.quarter
    print(f"Created date components for {col}")

In [ ]:
# 7.2 Create categories for high-cardinality columns (if needed)
# This is an example - adjust based on your specific categorical columns
for col in categorical_cols:
    if df_clean[col].nunique() > 100:  # High cardinality
        # Get top N categories and group the rest
        top_categories = df_clean[col].value_counts().nlargest(10).index
        df_clean[f"{col}_grouped"] = df_clean[col].apply(lambda x: x if x in top_categories else 'Other')
        print(f"Created grouped version of {col} with {df_clean[f'{col}_grouped'].nunique()} categories")

## 8. Data Standardization/Normalization

We'll standardize numerical features to ensure they're on the same scale.

In [ ]:
# Update numeric columns after feature engineering
numeric_cols = df_clean.select_dtypes(include=['int64', 'float64']).columns

# Create a new dataframe with standardized features
df_scaled = df_clean.copy()

# Apply standardization to numeric columns (excluding IDs and date components)
# Adjust the excluded columns based on your data
exclude_cols = [col for col in numeric_cols if 'ID' in col.upper() or '_year' in col or '_month' in col or '_quarter' in col]
scale_cols = [col for col in numeric_cols if col not in exclude_cols]

if scale_cols:  # Only proceed if there are columns to scale
    scaler = StandardScaler()
    df_scaled[scale_cols] = scaler.fit_transform(df_clean[scale_cols])
    print(f"Standardized {len(scale_cols)} numerical columns")

## 9. Encoding Categorical Variables

We'll encode categorical variables for use in modeling.

In [ ]:
# Update categorical columns list
categorical_cols = df_clean.select_dtypes(include=['object']).columns

# One-hot encode low-cardinality categorical variables
df_encoded = df_scaled.copy()
for col in categorical_cols:
    if df_clean[col].nunique() < 10:  # Only one-hot encode if few categories
        # Create dummy variables
        dummies = pd.get_dummies(df_clean[col], prefix=col, drop_first=True)
        # Add dummy variables to the dataframe
        df_encoded = pd.concat([df_encoded, dummies], axis=1)
        # Drop the original column
        df_encoded.drop(columns=[col], inplace=True)
        print(f"One-hot encoded {col} with {df_clean[col].nunique()} categories")

## 10. Final Data Validation

In [ ]:
# Check for any remaining missing values
remaining_missing = df_encoded.isnull().sum().sum()
print(f"Remaining missing values: {remaining_missing}")

# Check for any infinite values
infinite_values = np.isinf(df_encoded.select_dtypes(include=['float64', 'int64'])).sum().sum()
print(f"Infinite values: {infinite_values}")

# Basic shape of the cleaned dataset
print(f"Original dataset shape: {df.shape}")
print(f"Final cleaned dataset shape: {df_encoded.shape}")

## 11. Save Processed Dataset

In [ ]:
# Save the processed dataset to a CSV file
df_encoded.to_csv("datasets/building_permits_processed.csv", index=False)
print("Saved processed dataset to 'datasets/building_permits_processed.csv'")

## 12. Summary of Data Preparation Steps

In this notebook, we've completed a comprehensive data preparation process for the Building Permits dataset:

1. **Data Loading and Initial Inspection**:
   - Loaded the dataset with appropriate parameters
   - Examined data types, missing values, and basic statistics

2. **Data Cleaning**:
   - Fixed mixed-type columns
   - Converted date columns to datetime format
   - Handled missing values using appropriate strategies

3. **Outlier Detection and Treatment**:
   - Used Z-score method to identify outliers
   - Capped extreme values when appropriate

4. **Feature Engineering**:
   - Created date components from datetime columns
   - Grouped high-cardinality categorical variables

5. **Data Standardization**:
   - Standardized numerical features

6. **Categorical Encoding**:
   - One-hot encoded categorical variables with low cardinality

7. **Final Validation**:
   - Checked for any remaining issues
   - Verified the final dataset shape

8. **Saved Processed Dataset**:
   - Exported the cleaned and processed dataset

The processed dataset is now ready for exploratory data analysis, visualization, and modeling.